In [ ]:
import os.path as op
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
prefix = "/work/scratch/malinoro/histogram_transfer"

In [ ]:
img1 = cv.cvtColor(cv.imread(op.join(prefix, "conesF", "im2.ppm")), cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(cv.imread(op.join(prefix, "teddyF", "im2.ppm")), cv.COLOR_BGR2RGB)

## Fake image
X, Y = np.meshgrid(np.linspace(0, 255, img1.shape[0]), np.linspace(0, 255, img1.shape[1]))
R, G, B = np.sqrt((X**2+Y**2)/2), np.sqrt(((255-X)**2+Y**2)/2), np.sqrt(((255-X)**2+(255-Y)**2)/2)
R, G, B = R.astype(dtype=int), G.astype(dtype=int), B.astype(dtype=int) 
img0 = cv.merge([B, G, R])

In [ ]:
img = img0
bgr_planes = cv.split(img)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
axes[0].imshow(img)

n_bins = 80
for k, c in zip(range(3), ["b", "g", "r"]):
    counts, bins = np.histogram(bgr_planes[k], bins=n_bins, density=True)
    axes[1].stairs(counts, bins, color=c)
    axes[2].stairs(np.cumsum(counts), bins, baseline=None, color=c)
    
plt.show()

In [ ]:
cumulated_hist = True

bins_3d = (20, 20, 20)
X, Y, Z = np.meshgrid(*[np.linspace(0, 255, b) for b in bins_3d])


if cumulated_hist:
    counts, _ = np.histogramdd(np.array(img).reshape(-1, 3), bins=bins_3d)
    counts = counts.cumsum(axis=0).cumsum(axis=1).cumsum(axis=2) / np.sum(counts)
    iso_min, iso_max = 0, 1
else:
    counts, _ = np.histogramdd(np.array(img).reshape(-1, 3), bins=bins_3d, density=True)
    iso_min, iso_max = np.quantile(counts, 0.5), counts.max()

    
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    isomin=iso_min,
    isomax=iso_max,
    value=counts.flatten(),
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))

fig.update_layout(autosize=False, width=500, height=500, scene=dict(xaxis_title='Blue', yaxis_title='Green', zaxis_title='Red'))
fig.show()

### This proves that the histogram is correctly computed (raw format)